# Random Forest Training on CICIDS2017 Dataset

This notebook trains a Random Forest classifier on the CICIDS2017 intrusion detection dataset.

**Key Features:**
- SMOTE balancing applied within CV pipeline (prevents data leakage)
- Comprehensive preprocessing and feature engineering
- Data leakage diagnostics
- Feature importance analysis

## 1. Setup and Imports

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from CICIDS2017.preprocessing.dataset import CICIDS2017
from scripts.models.model_utils import (
    evaluate_model,
    check_data_leakage,
    get_feature_importance,
    remove_low_variance_features    
)

# Import model-specific modules
from scripts.models.random_forest import create_rf_pipeline, train_random_forest

from scripts.logger import LoggerManager

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful")

## 2. Initialize Logger

In [ ]:
logger = LoggerManager(log_name="rf_notebook").get_logger()
logger.info("Starting Random Forest training notebook")

## 3. Load and Preprocess Data

In [ ]:
# Load dataset
logger.info("Loading CICIDS2017 dataset...")
dataset = CICIDS2017(logger=logger)
dataset.encode().optimize_memory()
data = dataset.data

print(f"Dataset shape: {data.shape}")
print(f"\nColumns: {list(data.columns)}")
print(f"\nFirst few rows:")
data.head()

## 4. Sample Data

In [ ]:
# Sample data for faster training
SAMPLE_SIZE = 200000

logger.info(f"Sampling {SAMPLE_SIZE} rows from dataset...")
data_sample = data.sample(n=min(SAMPLE_SIZE, len(data)), random_state=0)

print(f"Sampled data shape: {data_sample.shape}")

## 5. Prepare Features and Labels

In [ ]:
# Split features and labels
X = data_sample.drop('Attack Type', axis=1)
y = data_sample['Attack Type']

# Remove known leakage features
leakage_features = ['Attack Number']  # Add other suspicious features here
existing_leakage = [f for f in leakage_features if f in X.columns]

if existing_leakage:
    logger.warning(f"🚨 REMOVING LEAKAGE FEATURES: {existing_leakage}")
    X = X.drop(columns=existing_leakage)

# Convert to numeric
X = X.apply(pd.to_numeric, errors='coerce')

# Handle missing values
if X.isnull().sum().sum() > 0:
    n_missing = X.isnull().sum().sum()
    logger.info(f"Filling {n_missing} missing values with 0")
    X = X.fillna(0)

print(f"Feature matrix shape: {X.shape}")
print(f"\nClass distribution:")
print(y.value_counts())

## 6. Visualize Class Distribution

In [ ]:
# Plot class distribution
plt.figure(figsize=(12, 6))
y.value_counts().plot(kind='bar')
plt.title('Class Distribution (Before SMOTE)', fontsize=14, fontweight='bold')
plt.xlabel('Attack Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Class imbalance ratio
class_counts = y.value_counts()
imbalance_ratio = class_counts.min() / class_counts.max()
print(f"\nClass imbalance ratio: {imbalance_ratio:.4f}")
print(f"Most common class: {class_counts.idxmax()} ({class_counts.max()} samples)")
print(f"Least common class: {class_counts.idxmin()} ({class_counts.min()} samples)")

## 7. Data Leakage Diagnostics

In [ ]:
# Check for potential data leakage
diagnostics = check_data_leakage(X, y, logger=logger)

# Display diagnostics
print("\n" + "="*50)
print("DATA LEAKAGE SUMMARY")
print("="*50)
print(f"Duplicates: {diagnostics['duplicates']} ({diagnostics['duplicate_pct']:.2f}%)")
print(f"High correlation features: {len(diagnostics['high_correlation_features'])}")
print(f"Constant features: {len(diagnostics['constant_features'])}")
print(f"Class balance ratio: {diagnostics['class_balance_ratio']:.4f}")

## 8. Remove Low Variance Features

In [ ]:
# Remove low variance features
X, removed_features = remove_low_variance_features(X, threshold=0.01, logger=logger)

print(f"\nRemoved {len(removed_features)} low variance features")
print(f"Remaining features: {X.shape[1]}")

## 9. Train/Test Split

In [ ]:
# Remove classes with fewer than 2 samples (needed for stratified split)
class_counts = y.value_counts()
valid_classes = class_counts[class_counts >= 2].index
X = X[y.isin(valid_classes)]
y = y[y.isin(valid_classes)]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=0,
    stratify=y
)

print(f"Train set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"\nTrain class distribution:")
print(y_train.value_counts())

## 10. Create Pipeline with SMOTE

In [ ]:
# Create pipeline with SMOTE
# SMOTE will be applied within each CV fold to prevent data leakage

pipeline = create_rf_pipeline(
    n_estimators=10,
    max_depth=3,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=0,
    use_smote=True,
    use_scaler=True,
    use_feature_selection=False,
    class_weight='balanced'
)

print("Pipeline created with steps:")
for name, step in pipeline.steps:
    print(f"  - {name}: {step.__class__.__name__}")

## 11. Cross-Validation

### What does Step 11 (Cross-Validation) do?

In this step, we use 5-fold cross-validation to estimate how well our Random Forest pipeline will generalize to new, unseen data. The process works by splitting the training data into 5 parts (folds), training the model on 4 parts, and validating it on the remaining part. This is repeated so each part is used as a validation set once. The average accuracy across all folds gives us a robust measure of model performance before we evaluate on the true test set.

**Why is this important?**
- It helps detect overfitting or underfitting.
- It allows us to compare different model settings fairly.
- It provides a realistic estimate of how the model will perform in practice, using only the training data.
- **Importantly, the real test set is never touched during cross-validation.** This ensures our final evaluation (in Step 13) is unbiased and reflects true out-of-sample performance.

In [ ]:
# Perform cross-validation
logger.info("Performing 5-fold cross-validation...")
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, n_jobs=-1)

print("\n" + "="*50)
print("CROSS-VALIDATION RESULTS")
print("="*50)
print(f"CV Scores: {cv_scores}")
print(f"Mean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

# Plot CV scores
plt.figure(figsize=(10, 6))
plt.plot(range(1, 6), cv_scores, marker='o', markersize=10, linewidth=2)
plt.axhline(y=cv_scores.mean(), color='r', linestyle='--', label=f'Mean: {cv_scores.mean():.4f}')
plt.xlabel('Fold', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Cross-Validation Scores', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 12. Train Final Model

In [ ]:
# Train on full training set
logger.info("Training final model on full training set...")
pipeline.fit(X_train, y_train)

print("✓ Model training completed")

## 13. Evaluate on Test Set

In [ ]:
# Evaluate model
results = evaluate_model(pipeline, X_test, y_test, logger=logger)

print("\n" + "="*50)
print("TEST SET RESULTS")
print("="*50)
print(f"Test Accuracy: {results['accuracy']:.4f}")
print(f"\nClassification Report:")
print(results['report'])

## 14. Confusion Matrix

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=results['confusion_matrix'],
                               display_labels=pipeline.classes_)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 15. Feature Importance Analysis

In [ ]:
# Get feature importance
top_features = get_feature_importance(
    pipeline,
    feature_names=list(X.columns),
    top_n=15,
    logger=logger
)

# Plot feature importance
features, importances = zip(*top_features)

plt.figure(figsize=(12, 8))
plt.barh(range(len(features)), importances)
plt.yticks(range(len(features)), features)
plt.xlabel('Importance', fontsize=12)
plt.title('Top 15 Feature Importances', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 16. Performance Summary

In [ ]:
# Summary
print("\n" + "="*70)
print("FINAL PERFORMANCE SUMMARY")
print("="*70)
print(f"Mean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
print(f"Test Accuracy: {results['accuracy']:.4f}")
print(f"\nModel Configuration:")
print(f"  - Number of trees: {pipeline.named_steps['rf'].n_estimators}")
print(f"  - Max depth: {pipeline.named_steps['rf'].max_depth}")
print(f"  - Max features: {pipeline.named_steps['rf'].max_features}")
print(f"  - SMOTE: Enabled")
print(f"  - Feature Scaling: Enabled")

# Performance indicators
if cv_scores.mean() > 0.99:
    print("\n⚠️  WARNING: CV score > 0.99 may indicate data leakage!")
    print("   Review feature engineering and data preprocessing.")
elif cv_scores.mean() >= 0.95:
    print("\n✓ Excellent performance achieved (CV score ≥ 0.95)")
elif cv_scores.mean() >= 0.90:
    print("\n✓ Good performance achieved (CV score ≥ 0.90)")
else:
    print("\n⚠️  Performance below 0.90")
    print("   Consider:")
    print("   - Increasing n_estimators or max_depth")
    print("   - Adding feature selection")
    print("   - Using more training data")
    print("   - Trying other algorithms (XGBoost, Neural Networks)")

logger.info("Notebook execution completed successfully!")

## 17. (Optional) Experiment with Different Configurations

In [ ]:
# Experiment with different configurations
configs = [
    {'name': 'Baseline', 'n_estimators': 100, 'max_depth': 10, 'use_feature_selection': False},
    {'name': 'More Trees', 'n_estimators': 200, 'max_depth': 10, 'use_feature_selection': False},
    {'name': 'Deeper Trees', 'n_estimators': 100, 'max_depth': 20, 'use_feature_selection': False},
    {'name': 'With Feature Selection', 'n_estimators': 150, 'max_depth': 15, 'use_feature_selection': True, 'n_features': 30},
]

results_comparison = []

for config in configs:
    print(f"\nTesting: {config['name']}")
    
    pipeline = create_rf_pipeline(
        n_estimators=config['n_estimators'],
        max_depth=config['max_depth'],
        use_smote=True,
        use_scaler=True,
        use_feature_selection=config.get('use_feature_selection', False),
        n_features=config.get('n_features', 30),
        random_state=0
    )
    
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, n_jobs=-1)
    mean_score = cv_scores.mean()
    
    results_comparison.append({
        'config': config['name'],
        'mean_cv_score': mean_score,
        'std_cv_score': cv_scores.std()
    })
    
    print(f"  Mean CV Score: {mean_score:.4f} (+/- {cv_scores.std():.4f})")

# Plot comparison
results_df = pd.DataFrame(results_comparison)

plt.figure(figsize=(12, 6))
plt.bar(results_df['config'], results_df['mean_cv_score'])
plt.errorbar(range(len(results_df)), results_df['mean_cv_score'], 
             yerr=results_df['std_cv_score'], fmt='none', color='black', capsize=5)
plt.xlabel('Configuration', fontsize=12)
plt.ylabel('Mean CV Score', fontsize=12)
plt.title('Configuration Comparison', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nBest configuration:")
best_idx = results_df['mean_cv_score'].idxmax()
print(results_df.iloc[best_idx])